![http://cognitiveatlas.org//static/img/logo-front.png](http://cognitiveatlas.org//static/img/logo-front.png)


<hr>

Here we are going to interact with the [Cognitive Atlas](https://www.cognitiveatlas.org) 
via it's [Python API](https://github.com/CognitiveAtlas/cogat-python).

## What is the Cognitive Atlas?

The Cognitive Atlas is a community developed knowledge base for concepts defined in cognitive psychology. More formally, this is called an ontology, and less formally, it's a place to look up what a term means that you see in the psychology literature, and how it relates to other terms.

We start using the cognitive atlas when we have a question of interest.

> What is working memory?

That seems like a good question! How might we have investigated the question, without any additional tool?

 - Do a Google Search and trust the internet
 - Do a literature review and try to synthesize many results
 - Rely on meta analysis
 
The above options are good, but each has drawbacks. For example, it would be hard to synthesize the opinions and studies from any of the resources above. Meta analyses are good, 

<hr>

## The Web Interface

If you are just exploring, you are best to interact with the Cognitive Atlas via it's web interface at [cognitiveatlas.org](https://www.cognitiveatlas.org). For example, here is a page for working memory at [http://cognitiveatlas.org/concept/id/trm_4a3fd79d0b5a7/](http://cognitiveatlas.org/concept/id/trm_4a3fd79d0b5a7/). I'm showing you the URL for a few important reasons:

  - we see that it's something called a "concept"
  - we see embedded in the URL is some kind of "id," the string `trm_4a3fd79d0b5a7`. Let's keep this in mind for later.

### Use Cases

From the interface we can quickly learn many things like the following:

### 1. Definition

We might be writing a review or paper, or just want to understand what the term means, to the best of the knowledge of the experts that have contributed to the Cognitive Atlas. Here we can see, front and center, this definition (here is the first bit of it).

> "active maintenance and flexible updating of goal/task relevant information (items, goals, strategies, etc.) in a form that resists interference but has limited capacity. ..."

This is great, but note that it hasn't been reviewed. We can do more investigation by looking at the [citations](http://cognitiveatlas.org/concept/id/trm_4a3fd79d0b5a7/#block-biblio) on the page, or in the case of a non Anonymous contribution, see the contribution author(s). Note that although you can't see it, there is a complete history kept of contributors to the terms.

### 2. Tasks

If you are a graduate student and wanting to conduct a study to measure working memory, this might be your stopping point! There is a list of over 15 [tasks](http://cognitiveatlas.org/concept/id/trm_4a3fd79d0b5a7/#block-measurements) on the page that have been asserted to measure working memory.

### 3. Relationships

How does working memory relate to the concept of [maintenance](http://cognitiveatlas.org/concept/id/trm_4a3fd79d0ba30/)? What about general [decision making](http://cognitiveatlas.org/concept/id/trm_4a3fd79d0a038/)? Remember that the Cognitive Atlas is an ontology, which means it defines both concepts **and** relationships between them! We have descriptors for these relationships, like "part of" or "kind of". You might see how this maps nicely to a graph:

 - a **concept** or **task** (or general entity represented in the atlas) maps to a node
 - a **relationship** between terms maps to a link between nodes
 - the strength of any relationship maps to a weight of a link between nodes
 
 This opens up a very powerful idea - we can use ideas and methods from graph theory to understand ideas in cognitive science. We can also generate visualizations to see them. For example, try posting this query:
 
 ```cypher
 MATCH (n:concept) where n.id = 'trm_4a3fd79d0ba30' OPTIONAL MATCH (n)-[]-(r) return n, r
 ```
Into the Neo4J browser (the top line of the window at [https://cognitiveatlas.org:7473/browser/](https://cognitiveatlas.org:7473/browser/)). You would see a little graph that describes the term "maintenance"

![neo4j-graph-maintenance.png](neo4j-graph-maintenance.png)

While many of these interactive, jumpy visualizations aren't super useful, they can be helpful for a talk, to show an idea in a paper, or solidify your understanding of a more complex relationship.

These are the basic functions and components of the Cognitive Atlas. If you are interested, you can also explore [Phenotypes](http://cognitiveatlas.org/disorders), [Tasks](http://cognitiveatlas.org/tasks/a/) and [Collections](http://cognitiveatlas.org/collections) (groups of tasks). But the web interface is limited in what we are able to analyze, in a large scale sense. In the next section, let's talk about more programmatic use of the Cognitive Atlas.

## The Cognitive Atlas API

API means "application programming interface" and it really is exactly that! It's a set of functions that you can call from a programming language (e.g., Python and R are common languages for scientific programming) to interact with a resource (for example, a database) to answer some question. In fact, APIs are essential for science - there are questions that, in order to answer, might require interaction with datasets beyond the ability of a human. We can't easily answer all queries that we might have about cognitive knowledge in a web browser! For example, here are some more advanced use cases that would require programming and (some) graph theory:

 - What cognitive concepts are represented across a database of over 1 million articles from the literature
 - **how** similar is working memory to all other concepts in the atlas?
     - Can you give me a number?
     - Can you define a threshold of significance?
 - What parts of the brain, represened by coordinate locations reported in the literature, are associated with the concept?

Beyond using the Cognitive Atlas as a lookup for terms, you can see that there are many use cases in research that would warrant interacting with it programatically, integrated with other tools, or scaled. Today let's explore one of these use cases - the third - and understand what areas of the brain might be involved with working memory.
 
# Getting Started Tutorial

The reason we are using a Jupyter Notebook is so that we can have code! If you are curious, this was generated with a Python 3 kernel, and installing the [Cognitive Atlas](https://github.com/CognitiveAtlas/cogat-python) Python API wrapper with pip:

```bash
pip install cognitiveatlas
```

Actually, the API itself is a [RESTful](https://codewords.recurse.com/issues/five/what-restful-actually-means) API, and could be interacted with via any programming language that can parse the [JSON endpoints](https://cognitiveatlas.org/api). Python is a popular language for neuroscience, and so it was chosen by the creator, who happens to be [this author](https://www.github.com/vsoch) :) It's a small world!

## Step 1. a question of interest

We don't want to approach a programming task without having clear definition of what we are aiming to do. Let's define a simple set of questions right now:

 1. What is working memory?
 2. What concepts are related to it?
 3. What parts of the brain have associations from the literature?
 
If we were using the interface we might have been okay going to the page discussed earlier, but we run into some trouble with the third question, because we need to integrate with another database (called [NeuroSynth](http://neurosynth.org) to derive a brain map, more on this later) and, while there actually is a button to drive this search now in the browser, we might want to easily do many of these queries, and for each, to download result maps at scale. That would be a lot of clicking to do in the interface! With programming, we can write a script and then go off and make a sandwich. Okay, so we have our dummy question of interest. Let's get started!

In [1]:
from cognitiveatlas.api import get_concept, get_task

In [ ]:

# EXAMPLE 1: #########################################################
# We are going to retrieve all cognitive paradgims (tasks), find 
# associated contrasts, and then find the cognitive concepts 
# the contrasts are asserted to measure. You can choose to do these 
# calls and arrange the data in whatever format fits your needs
######################################################################

# Step 1: Retrieve all tasks. When we do a general call for all tasks, we 
# actually only retrieve the basic information:
#
# [u'def_id_user',
# u'id',
# u'definition_text',
# u'event_stamp',
# u'def_event_stamp',
# u'concept_class',
# u'def_id',
# u'id_concept_class',
# u'id_user',
# u'name',
# u'type',
# u'alias']

#
# We will need a second call to get the rest, the addition of:
#[u'conclass',
# u'implementations',
# u'disorders',
# u'discussion',
# u'indicators',
# u'conditions',
# u'contrasts',
# u'external_datasets',
# u'umarkdef',
# u'umark',
# u'history']

# Step 2: Find contrasts associated with each task
# Note that this is an inefficient way to retrieve the full data, but it will work!

task_uids = [task["id"] for task in get_task().json]
contrasts = dict() # contrast lookup by task uid

# Now we can retrieve the full data. We are interested in contrasts, so let's save those.
for task in task_uids:
    if task not in contrasts:
        task_complete = get_task(task).json[0]
        # Only save if we have contrasts
        if len(task_complete["contrasts"]) > 0:
            contrasts[task] = task_complete["contrasts"]

# How many tasks have contrasts?
len(contrasts)
# 437

# Step 3: Make a contrast --> concept lookup
concepts = dict()
for task_uid,contrast_set in contrasts.iteritems():
    for contrast in contrast_set:
        contrast_uid = contrast["id"]
        if contrast_uid not in concepts:
            try: # Some calls don't work
                concepts[contrast_uid] = get_concept(contrast_id=contrast_uid).json[0]
            except:
                pass

# How many concepts are asserted to measure different contrasts?
len(concepts)
